<h1 style="text-align: center;">Simulación y aplicaciones en ciencias sociales y experimentales
</h3>
<h3 style="text-align: center;"> Tema 2.3. Diseño de experimentos </h3>

Una forma de medir la sensibilidad de un modelo a variaciones de los parámetros es por medio del **diseño de experimentos**. Estos consisten en asumir escenarios o **niveles** de los parámetros o **factores** del modelo y observar el efecto combinado de estos cambios sobre una variable objetivo, también denominada **respuesta**. 

Uno de los diseños de experimentos clásicos es el **diseño factorial $2^k$**, siendo $k$ el número de factores. Consiste en realizar las simulaciones para dos valores, notados por "-" y "+", de los dos parámetros.  



#### Caso de dos factores

En este caso la tabla de combinaciones es más compleja

| Combinación | Factor 1 ($\beta$) | Factor 2 ($\delta$) | Respuesta |
| --- | --- | --- |--- |
| 1 | - | - | $R_1$ |
| 2 | + | - | $R_2$ |
| 3 | - | + | $R_3$ |
| 4 | + | + | $R_4$ |

Entonces, el efecto medio del factor $j$ ($j=1,2$), es el cambio medio de la respuesta al pasar del signo $-$ al signo $+$, esto es: 

$$e_1 = {(R_2-R_1)+(R_4-R_3) \over 2},$$

$$e_2 = {(R_3-R_1)+(R_4-R_2) \over 2}.$$

Y el efecto de interacción $1 \times 2$, esto es, el efecto de un factor cuando se producen cambios en el otro factor, se calcula como: 

$$e_{12} = {1 \over 2} \left({(R_4-R_3) \over 1} - {(R_2-R_1) \over 1}\right),$$


#### Caso tres factores

Dado que tenemos tan solo dos factores, elegimos el **diseño factorial $2^k$**, siendo $k$ el número de factores. Consiste en realizar las simulaciones indicadas en la siguiente tabla: 

| Combinación | Factor 1 ($\beta$) | Factor 2 ($\delta$)| Factor 2 ($\gamma$) | Respuesta |
| --- | --- | --- |--- |--- |
| 1 | - | - |- | $R_1$ |
| 2 | + | - | - |$R_2$ |
| 3 | - | + |- | $R_3$ |
| 4 | + | + |- | $R_4$ |
| 5 | - | - |+ | $R_1$ |
| 6 | + | - | + |$R_2$ |
| 7 | - | + |+ | $R_3$ |
| 8 | + | + |+ | $R_4$ |

Entonces, el efecto medio del factor $j$ ($j=1,2,3$), es el cambio medio de la respuesta al pasar del signo $-$ al signo $+$, esto es: 

$$e_1 = {(R_2-R_1)+(R_4-R_3)+(R_6-R_5)+(R_8-R_7) \over 4},$$

$$e_2 = {(R_3-R_1)+(R_4-R_2)+(R_7-R_5)+(R_8-R_6) \over 4},$$

$$e_3 = {(R_5-R_1)+(R_6-R_2)+(R_7-R_3)+(R_8-R_4) \over 4}.$$

Y el efecto de interacción se calcula como: 

$$e_{12} = {1 \over 2} \left({(R_4-R_3)+(R_8-R_7) \over 2} - {(R_2-R_1)+(R_6-R_5) \over 2}\right),$$

$$e_{13} = {1 \over 2} \left({(R_6-R_5)+(R_8-R_7) \over 2} - {(R_2-R_1)+(R_4-R_3) \over 2}\right),$$

$$e_{23} = {1 \over 2} \left({(R_7-R_5)+(R_8-R_6) \over 2} - {(R_3-R_1)+(R_4-R_2) \over 2}\right),$$


En el modelo SIR con inmunización, realizar los siguientes análisis: 

**(a)** Estudiar el efecto de dos niveles de los parámetros $\beta$ y $\delta$.

**(b)** Estudiar el efecto de dos niveles de los parámetros $\beta$, $\gamma$ y $\delta$.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def sirv_model(beta, gamma, delta):
    def f(x, *_):
        s = x[0]
        i = x[1]
        ds = -beta*s*i -delta*s
        di = beta*s*i - gamma*i
        dr = gamma*i
        dv = delta*s
        return np.array([ds,di,dr,dv])
    return f

In [9]:
def solve_rk(model, x0, t):
    n = len(t)
    x = np.zeros((n, len(x0)))
    x[0] = x0
    for i in range(n-1):
        dt = t[i+1] - t[i]
        k1 = model(x[i], t[i])
        k2 = model(x[i]+dt*k1/2, t[i]+dt/2)
        k3 = model(x[i]+dt*k2/2, t[i]+dt/2)
        k4 = model(x[i]+dt*k3, t[i]+dt)
        x[i+1] = x[i] + dt/6 * (k1+2*k2+2*k3+k4) 
    return x

In [21]:
betas = (0.3, 0.5)
deltas = (0.01, 0.02)
gamma = 0.2

x0 = np.array([0.99, 0.01, 0, 0])
t = np.linspace(0, 100, 101)

In [37]:
params = {"beta": betas, "delta": deltas}
fixed = {"gamma": gamma}

def exp_design(params, fixed, model, x0, t, transf):
    pn = len(params)
    r_x = np.zeros(2**pn)
    for si in range(int(2**pn)):
        idxs = [(si>>i & 1) for i in range(pn)]
        mod_params = {
            name: ps[idx]
            for (name, ps), idx
            in zip(params.items(), idxs)
        }
        x = solve_rk(model(**fixed, **mod_params), x0, t)
        r_x[si] = transf(x)

    e_x = np.zeros(pn)
    for ei in range(pn):
        e_x[ei] = sum(
            r_i if 0 == (i>>ei & 1) else -r_i
            for i, r_i
            in enumerate(r_x)
        ) / 2**(pn-1)

    e_xx = np.zeros((pn, pn))
    for ei in range(pn):
        for ej in range(pn):
            if ei >= ej:
                continue
            e_xx[ei, ej] = sum(
                r_i if (i>>ei&1) == (i>>ej&1) else -r_i
                for i, r_i
                in enumerate(r_x)
            ) / 2**(pn-1)
    
    return {
        "r_x": r_x,
        "e_x": e_x,
        "e_xx": e_xx,
    }
        

In [41]:
params = {
    "beta": (0.3, 0.5),
    "gamma": (0.2, 0.4),
    "delta": (0.01, 0.02),
}

res = exp_design(params, {}, sirv_model, x0, t, lambda x: x[-1, 2])
res["e_xx"]

array([[ 0.        , -0.16324994, -0.02715588],
       [ 0.        ,  0.        ,  0.05105708],
       [ 0.        ,  0.        ,  0.        ]])

In [43]:
exp_design({
    "beta": (0.2, 0.4),
    "delta": (0.1, 0.12),
    "gamma": (0.1, 0.2),
}, {}, sirv_model, x0, t, lambda x: x[-1, 2])

{'r_x': array([0.030963  , 0.10727297, 0.02718744, 0.08124043, 0.02161111,
        0.05694801, 0.02023021, 0.04762304]),
 'e_x': array([-0.04827317,  0.01012849,  0.02506287]),
 'e_xx': array([[ 0.        , -0.00755026, -0.01690831],
        [ 0.        ,  0.        ,  0.00477555],
        [ 0.        ,  0.        ,  0.        ]])}

## Referencias

- Law, A.M. (2007). Simulation Modeling and Analysis. New York: McGraw Hill
